In [1]:
# Load packages
from cobra.io import read_sbml_model 
from cobra import Model, Reaction, Metabolite

In [2]:
# Load model
model = read_sbml_model('models/iMM904_progesterone.xml') # Load model

# Maximum theoretical productivity and yield of progesterone on default medium 

In order to assess the model, it can be beneficial to look at the maximum theoretical growth rate, as well as the productivity and yield obtained from the compound of interest. For this, we must first find the nutrients of the growth medium and the quantity of each nutrient.

In [3]:
# See medium compositions
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

It can be seen that the two limiting exchanges are glucose and oxygen, and that glucose is the sole carbon source of the default growth medium. 

We will now assess the maximum theoretical growth and the maximum theoretical progesterone production.

In [4]:
# Maximum growth and progesterone productivity/yield:
with model:
    baseline_max_growth = model.optimize().objective_value
    model.objective = model.reactions.DM_progesterone_c
    baseline_max_production = model.optimize().objective_value
    glucose_flux = model.reactions.EX_glc__D_e.flux
    baseline_max_yield = baseline_max_production/(-1 * glucose_flux)
print(f"Maximum growth: {round(baseline_max_growth,3)} gDW/h")
print(f"Maximum progesterone productivity: {round(baseline_max_production,3)} mmol/gDW*h")
print(f"Maximum progesterone yield: {round(baseline_max_yield,3)} mmol progesterone/mmol glucose")

Maximum growth: 0.288 gDW/h
Maximum progesterone productivity: 0.167 mmol/gDW*h
Maximum progesterone yield: 0.017 mmol progesterone/mmol glucose


The maximum theoretical growth is 0.288 gDW/h, the maximum theoretical productivity of progesterone is 0.167 mmol/gDW*h, and the maximum theoretical progesterone yield is 0.017 mmol progesterone/mmol glucose.

If the objective is biomass and cell growth, the cell produces none or very little progesterone. However, if the model objective is progesterone production, the cell growth will be zero (note that when the objective is set to the production of one compound, the biomass becomes zero as it is not viable for the cell). Therefore, we will simulate the maximum progesterone production when the objective contains both progesterone production and biomass. 

In [5]:
# Production for a dual model objective of BIOMASS and progesterone:
with model: 
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') 
    medium = model.medium
    model.objective = {model.reactions.get_by_id('BIOMASS_SC5_notrace'): baseline_max_production, 
                    model.reactions.get_by_id('DM_progesterone_c'): baseline_max_growth} # objective is to maximise the production of progesterone while also growing as much as possible
    solution = model.optimize()
    progesterone_production = solution.fluxes['DM_progesterone_c']
    glucose_flux = model.reactions.EX_glc__D_e.flux
    max_yield = progesterone_production/(-1 * glucose_flux)
    print('Glucose uptake:',format(medium[Glucose_exchange.id]), 'mmol/gDW*h')
    print('Maximum growth:', round(solution.fluxes['BIOMASS_SC5_notrace'],3), 'gDW/h')
    print('Maximum progesterone productivity:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield:', round(max_yield,3), 'mmol progesterone/mmol glucose') # calculate yield on galactose

Glucose uptake: 10.0 mmol/gDW*h
Maximum growth: 0.119 gDW/h
Maximum progesterone productivity: 0.156 mmol/gDW*h
Maximum progesterone yield: 0.016 mmol progesterone/mmol glucose


Using the default medium with glucose, the maximum theoretical growth is 0.119 gDW/h, the maximum theoretical productivity of progesterone is 0.156 mmol/gDW h and the maximum theoretical yield is 0.016 mmol progesterone/mmol glucose.

So now, what happens to progesterone production if we make more glucose available for uptake in the growth medium?

## Maximum theoretical productivity and yield in Cmole

To account for carbon molecules, instead of mass, we can convert the productivity and yield to Cmole.

In [6]:
# Define conversion factors
Cmole_glucose = 6
Cmole_progesterone = 21

In [7]:
# Productivity and yield in Cmole:
progesterone_production_Cmole = (progesterone_production)/(Cmole_progesterone) # Max productivity in Cmole
max_yield_Cmole = (max_yield*Cmole_progesterone)/(Cmole_glucose) # Max yield in Cmole
print('Maximum progesterone productivity:', round(progesterone_production_Cmole,5), 'Cmole/gDW*h') # Print max productivity in Cmole
print('Maximum progesterone yield:', round(max_yield_Cmole,5), 'Cmole progesterone/Cmole glucose') # Print max yield on glucose in Cmole

Maximum progesterone productivity: 0.00744 Cmole/gDW*h
Maximum progesterone yield: 0.05467 Cmole progesterone/Cmole glucose


# Increasing glucose availability in the growth medium

As seen before, one of the rate-limiting compounds is glucose. As glucose is only available in a low quantity in the medium, it could be interesting to increase the glucose concentration in the medium and see how it affects the theoretical growth rate of the strain.

In [8]:
# Simulate the model at different glucose uptake rates with biomass as objective:
with model:
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') # Define D-glucose exchange reaction
    medium = model.medium 
    objective_value = model.reactions.BIOMASS_SC5_notrace # Objective is to maximise growth
    growth = model.optimize().objective_value # Simulate model 
    print('Glucose uptake:', format(medium[Glucose_exchange.id]), 'mmol/gDW*h')
    print('Maximum growth:', round(growth,3), 'gDW/h') # since the objective is biomass, not flux, the unit changes to /h 
    print('')

    medium[Glucose_exchange.id] = 20
    model.medium = medium
    growth = model.optimize().objective_value
    print('Glucose uptake:', format(medium[Glucose_exchange.id]), 'mmol/gDW*h')
    print('Maximum growth:', round(growth,3), 'gDW/h')
    print('')

    medium[Glucose_exchange.id] = 30
    model.medium = medium
    growth = model.optimize().objective_value
    print('Glucose uptake:', format(medium[Glucose_exchange.id]), 'mmol/gDW*h')
    print('Maximum growth:', round(growth,3), 'gDW/h')



Glucose uptake: 10.0 mmol/gDW*h
Maximum growth: 0.288 gDW/h

Glucose uptake: 20 mmol/gDW*h
Maximum growth: 0.518 gDW/h

Glucose uptake: 30 mmol/gDW*h
Maximum growth: 0.739 gDW/h


The growth rate clearly increases in parallel with the availability of glucose in the growth medium. Hence, it could be interesting to simulate the effect of increasing glucose in the medium on progesterone production.

## The effect on progesterone production by increasing glucose availability

Now, we will simulate the effect that increasing the glucose concentration in the medium has on the production of progesterone.

In [9]:
# Simulate the model with increasing glucose uptake with progesterone production as objective:
with model: 
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') 
    medium = model.medium
    
    model.objective = {model.reactions.get_by_id('BIOMASS_SC5_notrace'): baseline_max_production, 
                    model.reactions.get_by_id('DM_progesterone_c'): baseline_max_growth}
    solution = model.optimize() # simulate model
    progesterone_production = solution.fluxes['DM_progesterone_c']
    glucose_flux = model.reactions.EX_glc__D_e.flux
    max_yield = progesterone_production/(-1 * glucose_flux)
    print('Glucose uptake:',format(medium[Glucose_exchange.id]),'mmol/gDW*h')
    print('Maximum growth:', round(solution.fluxes['BIOMASS_SC5_notrace'],3), 'gDW/h')
    print('Maximum progesterone productivity on glucose:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield on glucose:', round(max_yield,3), 'mmol progesterone/mmol glucose') # calculate yield on galactose
    print('')

    medium[Glucose_exchange.id] = 20 # change glucose uptake
    model.medium = medium 

    model.objective = {model.reactions.get_by_id('BIOMASS_SC5_notrace'): baseline_max_production, 
                    model.reactions.get_by_id('DM_progesterone_c'): baseline_max_growth}
    solution = model.optimize() # simulate model
    progesterone_production = solution.fluxes['DM_progesterone_c']
    glucose_flux = model.reactions.EX_glc__D_e.flux
    max_yield = progesterone_production/(-1 * glucose_flux)
    print('Glucose uptake:',format(medium[Glucose_exchange.id]),'mmol/gDW*h')
    print('Maximum growth:', round(solution.fluxes['BIOMASS_SC5_notrace'],3), 'gDW/h')
    print('Maximum progesterone productivity on glucose:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield on glucose:', round(max_yield,3), 'mmol progesterone/mmol glucose') # calculate yield on galactose
    print('')

    medium[Glucose_exchange.id] = 30 # change glucose uptake
    model.medium = medium 

    model.objective = {model.reactions.get_by_id('BIOMASS_SC5_notrace'): baseline_max_production, 
                    model.reactions.get_by_id('DM_progesterone_c'): baseline_max_growth}
    solution = model.optimize() # simulate model
    progesterone_production = solution.fluxes['DM_progesterone_c']
    glucose_flux = model.reactions.EX_glc__D_e.flux
    max_yield = progesterone_production/(-1 * glucose_flux)
    print('Glucose uptake:',format(medium[Glucose_exchange.id]),'mmol/gDW*h')
    print('Maximum growth:', round(solution.fluxes['BIOMASS_SC5_notrace'],3), 'gDW/h')
    print('Maximum progesterone productivity on glucose:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield on glucose:', round(max_yield,3), 'mmol progesterone/mmol glucose') # calculate yield on galactose

Glucose uptake: 10.0 mmol/gDW*h
Maximum growth: 0.119 gDW/h
Maximum progesterone productivity on glucose: 0.156 mmol/gDW*h
Maximum progesterone yield on glucose: 0.016 mmol progesterone/mmol glucose

Glucose uptake: 20 mmol/gDW*h
Maximum growth: 0.342 gDW/h
Maximum progesterone productivity on glucose: 0.153 mmol/gDW*h
Maximum progesterone yield on glucose: 0.008 mmol progesterone/mmol glucose

Glucose uptake: 30 mmol/gDW*h
Maximum growth: 0.565 gDW/h
Maximum progesterone productivity on glucose: 0.15 mmol/gDW*h
Maximum progesterone yield on glucose: 0.005 mmol progesterone/mmol glucose


When increasing glucose availability in the medium, the growth rate increases. Interestingly, the productivity of progesterone only changes slightly as the amount of glucose increases. Since the productivity changes only slightly when the glucose concentration increases, the yield is lowered. Therefore, only increasing glucose concentration is not a feasible idea for increasing progesterone yield - model changes must be made to increase the productivity of progesterone. Only then, increasing the glucose concentration might be beneficial. 

## Maximum theoretical productivity and yield of progesterone on alternative carbon sources

The default medium is not necesseraily the most efficient for the growth of our strain and the production of progesterone. Therefore, we will now simulate what happens, if the carbon source is shifted from glucose to galactose and fructose, respectively.

In [10]:
# Growth and progesterone yield/productivity on galactose:
with model: #galactose
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') #define glucose reaction
    Galactose_exchange = model.reactions.get_by_id('EX_gal_e') #define galactose reaction
    medium = model.medium
    print('Medium with glucose (before):', model.medium)
    print('')
    medium[Glucose_exchange.id] = 0 # Set glucose to zero
    model.medium = medium
    medium[Galactose_exchange.id] = 10 # add galactose to medium
    model.medium = medium
    print('Medium with galactose (after):',model.medium)
    print() 
    print('When objective is set to only progesterone production:')
    model.objective = model.reactions.R02216 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    solution1 = model.optimize()
    solution = model.optimize().objective_value
    progesterone_production = solution
    galactose_flux = model.reactions.EX_gal_e.flux
    max_yield_gal = progesterone_production/(-1 * galactose_flux)
    print('Galactose uptake:',format(medium[Galactose_exchange.id]),'mmol/gDW*h')
    print('Maximum growth:', round(solution1.fluxes['BIOMASS_SC5_notrace'],3), 'gDW/h')
    print('Maximum progesterone productivity on galactose:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield on galactose:', round(max_yield_gal,3), 'mmol progesterone/mmol galactose') # calculate yield on galactose

with model: #galactose
    print()
    print('When objective is set at both growth and progesterone production:')
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') #define glucose reaction
    Galactose_exchange = model.reactions.get_by_id('EX_gal_e') #define galactose reaction
    medium = model.medium
    #print('Medium before:', model.medium)
    #print('')
    medium[Glucose_exchange.id] = 0 # Set glucose to zero
    model.medium = medium
    medium[Galactose_exchange.id] = 10 # add galactose to medium
    model.medium = medium
    #print('Medium after:',model.medium)
    #print('') 
    model.objective = {model.reactions.get_by_id('BIOMASS_SC5_notrace'): baseline_max_production, 
                    model.reactions.get_by_id('DM_progesterone_c'): baseline_max_growth} # objective is to maximise the production of progesterone while also growing as much as possible
    solution = model.optimize()
    progesterone_production = solution.fluxes['DM_progesterone_c']
    galactose_flux = model.reactions.EX_gal_e.flux
    max_yield_gal = progesterone_production/(-1 * galactose_flux)
    print('Galactose uptake:',format(medium[Galactose_exchange.id]),'mmol/gDW*h')
    print('Maximum growth:', round(solution.fluxes['BIOMASS_SC5_notrace'],3), 'gDW/h')
    print('Maximum progesterone productivity on galactose:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield on galactose:', round(max_yield_gal,3), 'mmol progesterone/mmol galactose') # calculate yield on galactose

Medium with glucose (before): {'EX_fe2_e': 999999.0, 'EX_glc__D_e': 10.0, 'EX_h2o_e': 999999.0, 'EX_h_e': 999999.0, 'EX_k_e': 999999.0, 'EX_na1_e': 999999.0, 'EX_so4_e': 999999.0, 'EX_nh4_e': 999999.0, 'EX_o2_e': 2.0, 'EX_pi_e': 999999.0}

Medium with galactose (after): {'EX_fe2_e': 999999.0, 'EX_gal_e': 10, 'EX_h2o_e': 999999.0, 'EX_h_e': 999999.0, 'EX_k_e': 999999.0, 'EX_na1_e': 999999.0, 'EX_so4_e': 999999.0, 'EX_nh4_e': 999999.0, 'EX_o2_e': 2.0, 'EX_pi_e': 999999.0}

When objective is set to only progesterone production:
Galactose uptake: 10 mmol/gDW*h
Maximum growth: 0.0 gDW/h
Maximum progesterone productivity on galactose: 0.167 mmol/gDW*h
Maximum progesterone yield on galactose: 0.017 mmol progesterone/mmol galactose

When objective is set at both growth and progesterone production:
Galactose uptake: 10 mmol/gDW*h
Maximum growth: 0.119 gDW/h
Maximum progesterone productivity on galactose: 0.156 mmol/gDW*h
Maximum progesterone yield on galactose: 0.016 mmol progesterone/mmol gala

The yield of progesterone increases when the objective is progesterone production. However, if the model objective is both growth and progesterone production, it is not increased when galactose is used as carbon source over glucose.

In [11]:
# Growth and progesterone yield/productivity on fructose:
with model: # fructose
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') #define glucose reaction
    Fructose_exchange = model.reactions.get_by_id('EX_fru_e') #define fructose reaction
    medium = model.medium
    print('Medium before:', model.medium)
    print('')
    medium[Glucose_exchange.id] = 0 # Set D-glucose to zero
    model.medium = medium
    medium[Fructose_exchange.id] = 10 # add fructose to medium
    model.medium = medium 
    print('Medium after:',model.medium)
    print('')
    print('When objective is set to only progesterone production:')
    model.objective = model.reactions.R02216 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    solution1 = model.optimize()
    solution = model.optimize().objective_value # define new model with maximum production of progesterone
    progesterone_production = solution
    fructose_flux = model.reactions.EX_fru_e.flux
    max_yield_fru = progesterone_production/(-1 * fructose_flux)
    print('Fructose uptake:', format(medium[Fructose_exchange.id]),'mmol/gDW*h')
    print('Maximum growth:', round(solution1.fluxes['BIOMASS_SC5_notrace'],3), 'gDW/h')
    print('Maximum progesterone productivity on fructose:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield on fructose:', round(max_yield_fru,3), 'mmol progesterone/mmol fructose') # calculate yield on fructose

with model: # fructose
    print('')
    print('When objective is set at both growth and progesterone production:')
    Glucose_exchange = model.reactions.get_by_id('EX_glc__D_e') #define glucose reaction
    Fructose_exchange = model.reactions.get_by_id('EX_fru_e') #define fructose reaction
    medium = model.medium
    #print('Medium before:', model.medium)
    #print('')
    medium[Glucose_exchange.id] = 0 # Set D-glucose to zero
    model.medium = medium
    medium[Fructose_exchange.id] = 10 # add fructose to medium
    model.medium = medium 
    #print('Medium after:',model.medium)
    #print('')
    model.objective = {model.reactions.get_by_id('BIOMASS_SC5_notrace'): baseline_max_production, 
                    model.reactions.get_by_id('DM_progesterone_c'): baseline_max_growth} # objective is to maximise the production of progesterone while also growing as much as possible
    solution = model.optimize() # define new model with maximum production of progesterone
    progesterone_production = solution.fluxes['DM_progesterone_c']
    fructose_flux = model.reactions.EX_fru_e.flux
    max_yield_fru = progesterone_production/(-1 * fructose_flux)
    print('Fructose uptake:', format(medium[Fructose_exchange.id]),'mmol/gDW*h')
    print('Maximum growth:', round(solution.fluxes['BIOMASS_SC5_notrace'],3), 'gDW/h')
    print('Maximum progesterone productivity on fructose:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield on fructose:', round(max_yield_fru,3), 'mmol progesterone/mmol fructose') # calculate yield on fructose

Medium before: {'EX_fe2_e': 999999.0, 'EX_glc__D_e': 10.0, 'EX_h2o_e': 999999.0, 'EX_h_e': 999999.0, 'EX_k_e': 999999.0, 'EX_na1_e': 999999.0, 'EX_so4_e': 999999.0, 'EX_nh4_e': 999999.0, 'EX_o2_e': 2.0, 'EX_pi_e': 999999.0}

Medium after: {'EX_fe2_e': 999999.0, 'EX_fru_e': 10, 'EX_h2o_e': 999999.0, 'EX_h_e': 999999.0, 'EX_k_e': 999999.0, 'EX_na1_e': 999999.0, 'EX_so4_e': 999999.0, 'EX_nh4_e': 999999.0, 'EX_o2_e': 2.0, 'EX_pi_e': 999999.0}

When objective is set to only progesterone production:
Fructose uptake: 10 mmol/gDW*h
Maximum growth: 0.0 gDW/h
Maximum progesterone productivity on fructose: 0.167 mmol/gDW*h
Maximum progesterone yield on fructose: 0.017 mmol progesterone/mmol fructose

When objective is set at both growth and progesterone production:
Fructose uptake: 10 mmol/gDW*h
Maximum growth: 0.119 gDW/h
Maximum progesterone productivity on fructose: 0.156 mmol/gDW*h
Maximum progesterone yield on fructose: 0.016 mmol progesterone/mmol fructose


When the objective is set only at improving progesterone production, fructose and galactose obtain the exact same yield as when using glucose (0.017); if the model objective is both growth and progesterone production, the same conclusion applies. In conclusion, no carbon source can utilize a better pathway and increase cell productivity over the default media containing glucose. 